# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('../WeatherPy/Cities_data.csv', encoding="ISO-8859-1")

weather_df.dtypes

City                 object
Lattitude           float64
Longitude           float64
Temp [F]            float64
Humidity [%]          int64
Cloudiness [%]        int64
Wind Speed (mph)    float64
Country              object
Date                  int64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = weather_df[['Lattitude', 'Longitude']]
humidity = weather_df['Humidity [%]']

fig = gmaps.figure()

#Figure out how to make the scale more understandable here
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, point_radius=1, max_intensity=max(humidity))

fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
ideal_df = weather_df.loc[(weather_df['Temp [F]'] <= 80) & (weather_df['Temp [F]'] >= 70) & (weather_df['Wind Speed (mph)'] < 10) & (weather_df['Cloudiness [%]'] == 0)]

# len(ideal_df)

ideal_df.head()

,City,Lattitude,Longitude,Temp [F],Humidity [%],Cloudiness [%],Wind Speed (mph),Country,Date
34,port macquarie,-31.4333,152.9167,73.99,42,0,1.99,AU,1610749698
134,saint-pierre,-21.3393,55.4781,78.80,89,0,2.30,RE,1610749711
137,namibe,-15.1961,12.1522,74.32,68,0,8.01,AO,1610749712
173,masterton,-40.9597,175.6575,71.01,80,0,3.00,NZ,1610749716
194,saint-philippe,-21.3585,55.7679,78.80,89,0,2.30,RE,1610749719


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
hotel_df = ideal_df[['Lattitude', 'Longitude','City', 'Country']].copy()
hotel_df['Hotel Name'] = ''


params = {
    "radius": 5000,
    "type": 'lodging',
    "key": g_key
}

for index,row in hotel_df.iterrows(): #['-31.4333,152.9167']
    
#     print(index)
#     print(row)
    lat = row['Lattitude']
    long = row['Longitude']
    params['location'] = f'{lat},{long}'
    
    
    place_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotels = requests.get(place_url, params=params).json()
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except(KeyError, IndexError):
        print('skipping row')
        
        
hotel_df.head()
        

,Lattitude,Longitude,City,Country,Hotel Name
34,-31.4333,152.9167,port macquarie,AU,Rydges Port Macquarie
134,-21.3393,55.4781,saint-pierre,RE,Lindsey Hôtel
137,-15.1961,12.1522,namibe,AO,Hotel Chik Chik Namibe
173,-40.9597,175.6575,masterton,NZ,Copthorne Hotel & Resort Solway Park Wairarapa
194,-21.3585,55.7679,saint-philippe,RE,"Chambres d'hôte ""La Trinité"""


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lattitude", "Longitude"]]

In [24]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))